In [1]:
import os
from dask_gateway import Gateway
# gateway = Gateway()
gateway = Gateway(
    "http://dask-gateway-k8s.geddes.rcac.purdue.edu/",
    proxy_address="traefik-dask-gateway-k8s.cms.geddes.rcac.purdue.edu:8786",
)

In [2]:
os.environ["X509_USER_PROXY"] = "/depot/cms/users/yun79/x509up_u383571"
os.environ["PYTHONPATH"] += ":/depot/cms/private/users/yun79/valerie/fork/copperheadV2"
os.environ['XRD_REQUESTTIMEOUT'] = "2400"
cluster = gateway.new_cluster(
    conda_env = "/depot/cms/kernels/coffea2024",
    worker_cores = 1,#1
    worker_memory = 4,
    env = dict(os.environ),
)

cluster.scale(100) 
cluster

In [2]:
os.environ['PWD']

'/depot/cms/private/users/yun79/valerie/fork/copperheadV2'

In [3]:
"""
To shutdown the cluster, run this cell
Equivalently, you can also do "squeue -u <username>" 
if this is the only slurm job you're doing
"""

for cluster_info in gateway.list_clusters():
    gateway.connect(cluster_info.name).shutdown()

In [14]:
options = gateway.cluster_options()  

In [ ]:
# options.worker_cores
options.fields

In [9]:
! squeue -u yun79

':/depot/cms/purdue-af/lhapdf/lib/python3.10/site-packages'